In [ ]:
from prefect import flow, task
from prefect_dask import DaskTaskRunner


#only works when running on slac ip
# prefect config set PREFECT_API_URL="https://ard-modeling-service.slac.stanford.edu/api" 

# Define tasks
@task
def fetch_data(dataset: str) -> str:
    """
    Simulate fetching data for a given dataset.
    """
    print(f"Fetching data for {dataset}")
    return f"data_{dataset}"

@task
def process_data(data: str) -> str:
    """
    Simulate processing of data.
    """
    print(f"Processing {data}")
    return f"processed_{data}"

@task
def save_results(result: str) -> None:
    """
    Simulate saving processed data.
    """
    print(f"Saving {result}")

# Define flow
@flow(task_runner=DaskTaskRunner())
def parallel_workflow(datasets: list):
    """
    A workflow that fetches, processes, and saves datasets in parallel.
    """
    # Fetch data in parallel
    raw_data = [fetch_data.submit(dataset) for dataset in datasets]
    
    # Process data in parallel
    processed_data = [process_data.submit(data) for data in raw_data]
    
    # Save results in parallel
    [save_results.submit(result) for result in processed_data]

# Execute the workflow
if __name__ == "__main__":
    # List of datasets to process
    datasets = ["dataset1", "dataset2", "dataset3", "dataset4"]
    parallel_workflow(datasets)